### Дано:
Таблица active_clients содержит ежемесячный срез клиентов банка, которые совершали какие-либо транзакции в данном месяце.
Поля: отчетный месяц (report_month) и идентификатор клиента (client_id).
Считаем, что клиент "оттек" из банка в месяце N, если в месяце N он активен (присутсвует в таблице active_clients) и не активен в месяцы N+1, N+2, N+3.
### Задание:
Вывести количество активных клиентов на каждый месяц и долю клиентов, которые "оттекли" в каждом месяце.

In [1]:
import pandas as pd

column_report_month = [
    '2018-01-01', '2018-01-01', '2018-01-01', '2018-02-01', '2018-02-01', '2018-03-01','2018-03-01',
    '2018-04-01', '2018-04-01', '2018-04-01', '2018-05-01', '2018-05-01', '2018-05-01'
]
column_client_id = [
    '1', '2', '3', '2', '4', '1', '2', '1', '2', '4', '1', '5', '2'
]
active_clients = pd.DataFrame(columns=['report_month', 'client_id'])
active_clients['report_month'] = column_report_month
active_clients['report_month'] = pd.to_datetime(active_clients['report_month'])
active_clients['client_id'] = column_client_id
active_clients['client_id'] = active_clients['client_id'].astype('int32')

In [2]:
# Копия датасета с сортировкой по клиентам и месяцам
active_clients_month = active_clients[['client_id','report_month']] \
    .sort_values(['client_id','report_month']) \
    .reset_index(drop=True) \
    .copy(deep=False)

In [3]:
# Добавить столбец со следующим активным месяцем
active_clients_month['till_next_active_month'] = active_clients_month \
    .groupby(['client_id'])['report_month'].shift(-1)

In [4]:
# Замена NaT на слудущий месяц после максимальной даты
active_clients_month['till_next_active_month'] \
    .fillna(active_clients['report_month'].max() + pd.DateOffset(months=1), inplace=True)

/tmp/ipykernel_12582/1378778464.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  active_clients_month['till_next_active_month'] \


In [5]:
# Добавить столбец с количестом месяцев до следующей активности
active_clients_month['count_of_months_until_next_activity'] = \
    ((active_clients_month['till_next_active_month'] - active_clients_month['report_month'])/28) \
    .dt.days

In [6]:
# Удаляем данные за последние три месяца
m3 = active_clients["report_month"].max() - pd.DateOffset(months=3)
active_clients_month = active_clients_month.query('report_month <= @m3')

In [7]:
# Количество активных клиентов по месяцам
count_clients_by_month = active_clients_month \
    .groupby(['report_month'], as_index=False) \
    .agg({'client_id': 'count'}) \
    .reset_index(drop=True) \
    .copy(deep=False)
count_clients_by_month.rename(columns={'client_id': 'active_clients'}, inplace=True)

In [8]:
# Количество оттекших клиентов по месяцам
churn_clietns_by_month = active_clients_month \
    .query('count_of_months_until_next_activity >= 3') \
    .groupby(['report_month'], as_index=False) \
    .agg({'client_id': 'count'}) \
    .reset_index(drop=True) \
    .copy(deep=False)
churn_clietns_by_month.rename(columns={'client_id': 'chur_clients'}, inplace=True)

In [9]:
# Добавить через левый джоин количество оттекших клиентов
count_clients_by_month = count_clients_by_month.merge(churn_clietns_by_month, how='left', on='report_month')

In [10]:
# Добавить долю оттёкших
count_clients_by_month['churn_rate'] = count_clients_by_month['chur_clients']/count_clients_by_month['active_clients']
count_clients_by_month.fillna(0, inplace=True)

In [11]:
# Количество активных клиентов и доля оттёкших
churn_rate_by_month = count_clients_by_month[['report_month', 'active_clients', 'churn_rate']]
churn_rate_by_month

,report_month,active_clients,churn_rate
0,2018-01-01,3,0.333333
1,2018-02-01,2,0.000000
